In [ ]:
import sys
sys.path.append('..')

from pathlib import Path
from dataclasses import dataclass

from PIL import Image
import matplotlib.pyplot as plt
from tqdm import trange

import torch
import torchvision.transforms as T
from diffusers import StableDiffusionImg2ImgPipeline

from arch import utils

In [ ]:
DEVICE = utils.device_mapper()
print(f"Device: {str(DEVICE).upper()}")

In [ ]:
pipeline = StableDiffusionImg2ImgPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", variant='fp16', torch_dtype=torch.float16)
pipeline = pipeline.to(DEVICE)

In [ ]:
PATH = Path('data/puppy.png')
init_image = utils.load_image(PATH)
init_image

In [ ]:
def preprocess(img):
    w, h = map(lambda x: x - x % 32, img.size)  # Resize to integer multiple of 32
    img = img.resize((w, h), resample=Image.LANCZOS)
    img = utils.image2array(img)
    img = img[None].transpose(0, 3, 1, 2)
    img = torch.from_numpy(img)
    return 2.0 * img - 1.0

In [ ]:
def pgd_attack(x, model, eps=0.1, step_size=0.015, iters=40):
    x_adv = x.clone().detach() + (torch.rand(*x.shape) * 2 * eps-eps).to(DEVICE)

    for i in (t := trange(iters)):
        actual_step_size = step_size - (step_size - step_size / 100) / iters * i  

        x_adv.requires_grad_(True)
        loss = (model(x_adv).latent_dist.mean).norm()

        t.set_description(f"[Running attack]: Loss {loss.item():.5f} | step size: {actual_step_size:.4}")

        grad, = torch.autograd.grad(loss, [x_adv])
        x_adv = x_adv - grad.detach().sign() * actual_step_size
        x_adv = torch.minimum(torch.maximum(x_adv, x - eps), x + eps)
        x_adv.data = torch.clamp(x_adv, min=-1, max=1)
        x_adv.grad = None

    return x_adv

In [ ]:
# eps | Higher = Less imperceptible
# step | Set smaller than eps
# iters | Higher = Stronger

with torch.autocast(str(DEVICE)):
    X = preprocess(init_image).half().to(DEVICE)
    adv_X = pgd_attack(X, model=pipeline.vae.encode, eps=0.06, step_size=0.02, iters=1000)
    adv_X = (adv_X / 2 + 0.5).clamp(0, 1)

In [ ]:
toPIL = T.ToPILImage()

adv_image = toPIL(adv_X[0]).convert("RGB")
adv_image

In [ ]:
@dataclass
class HyperConfig:
    prompt = "dog under heavy rain and muddy ground real"
    strength = 0.5
    infer_steps = 50
    cfg_scale = 7.0
    seed = 9222

config = HyperConfig()

In [ ]:
with torch.autocast('cuda'):
    image_nat = pipeline(prompt=config.prompt, 
                         image=init_image, 
                         strength=config.strength, 
                         guidance_scale=config.cfg_scale, 
                         num_inference_steps=config.infer_steps).images[0]
    
    image_adv = pipeline(prompt=config.prompt, 
                         image=adv_image, 
                         strength=config.strength, 
                         guidance_scale=config.cfg_scale, 
                         num_inference_steps=config.infer_steps).images[0]


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(20,6))

ax[0].imshow(init_image)
ax[1].imshow(adv_image)
ax[2].imshow(image_nat)
ax[3].imshow(image_adv)

ax[0].set_title('Source Image', fontsize=12)
ax[1].set_title('Adv Image', fontsize=12)
ax[2].set_title('Gen. Image Nat.', fontsize=12)
ax[3].set_title('Gen. Image Adv.', fontsize=12)

for i in range(4):
    ax[i].grid(False), ax[i].axis('off')

fig.tight_layout()